In [1]:
import mlflow
import optax
import pandas as pd
import polars as pl
from flax import nnx
from flax_trainer.evaluator import RegressionEvaluator
from flax_trainer.loader import MiniBatchLoader
from flax_trainer.loss_fn import mean_squared_error
from flax_trainer.trainer import Trainer
from sklearn.model_selection import train_test_split

from nn_recsys.model.two_tower import TwoTower

### READ

In [2]:
rating_df, movie_df, user_df = (
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/ratings.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["user_id", "item_id", "rating", "timestamp"],
        )
    ),
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/movies.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["item_id", "title", "genres"],
            encoding="ISO-8859-1",
        )
    ).with_columns(pl.col("genres").str.split("|")),
    pl.from_pandas(
        pd.read_csv(
            "../../dataset/ML1M/ml-1m/users.dat",
            delimiter="::",
            engine="python",
            header=None,
            names=["user_id", "gender", "age", "occupation", "zip_code"],
            # encoding="ISO-8859-1",
        )
    ),
)

dataset_df = (
    rating_df.join(movie_df, how="left", on="item_id")
    .join(user_df, how="left", on="user_id")
    .select(pl.all().exclude("rating", "genres"), pl.col("rating"))
    .with_columns(pl.all().exclude("rating").rank("dense") - 1)
)

user_categorical_feature_indices = [0, 4, 5, 6, 7]
user_categorical_feature_cardinalities = [
    dataset_df[:, idx].n_unique() for idx in user_categorical_feature_indices
]
user_numerical_feature_indices = []
item_categorical_feature_indices = [1, 3]
item_categorical_feature_cardinalities = [
    dataset_df[:, idx].n_unique() for idx in item_categorical_feature_indices
]
item_numerical_feature_indices = []

train_df, valid_df = train_test_split(
    dataset_df, test_size=0.1, random_state=0, shuffle=True
)
dataset_df

user_id,item_id,timestamp,title,gender,age,occupation,zip_code,rating
u32,u32,u32,u32,u32,u32,u32,u32,i64
0,1104,397442,2452,0,0,10,1588,5
0,639,397457,1739,0,0,10,1588,3
0,853,397454,2289,0,0,10,1588,3
0,3177,397440,1054,0,0,10,1588,4
0,2162,400250,557,0,0,10,1588,5
…,…,…,…,…,…,…,…,…
6039,1019,382,3574,1,2,6,466,1
6039,1022,21,814,1,2,6,466,5
6039,548,17,3578,1,2,6,466,5


### 学習

In [3]:
model = TwoTower(
    user_categorical_feature_indices=user_categorical_feature_indices,
    user_categorical_feature_cardinalities=user_categorical_feature_cardinalities,
    user_numerical_feature_indices=user_numerical_feature_indices,
    user_hidden_layer_dims=[128, 64],
    item_categorical_feature_indices=item_categorical_feature_indices,
    item_categorical_feature_cardinalities=item_categorical_feature_cardinalities,
    item_numerical_feature_indices=item_numerical_feature_indices,
    item_hidden_layer_dims=[128, 64],
    embed_dim=30,
    output_layer_dim=30,
    rngs=nnx.Rngs(0),
)

In [4]:
loader = MiniBatchLoader(dataset_df=train_df, batch_size=512, rngs=nnx.Rngs(0))
evaluator = RegressionEvaluator(dataset_df=valid_df)

In [5]:
mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("TWO_TOWER_RATING")

with mlflow.start_run() as run:
    trainer = Trainer(
        model=model,
        optimizer=optax.adamw(learning_rate=0.001, weight_decay=0.001),
        train_loader=loader,
        loss_fn=mean_squared_error,
        valid_evaluator=evaluator,
        early_stopping_patience=10,
        epoch_num=32,
        active_run=run,
    )
    trainer = trainer.fit()

[VALID 000]: loss=14.048372268676758, metrics={'mse': 14.048372268676758}


[TRAIN 001]: 100%|██████████| 1759/1759 [00:10<00:00, 168.68it/s, batch_loss=0.7955281] 


[VALID 001]: loss=0.8363550901412964, metrics={'mse': 0.8363550901412964}


[TRAIN 002]: 100%|██████████| 1759/1759 [00:13<00:00, 133.04it/s, batch_loss=0.83432454]


[VALID 002]: loss=0.8051806688308716, metrics={'mse': 0.8051806688308716}


...

[VALID 015]: loss=0.785104513168335, metrics={'mse': 0.785104513168335}


[TRAIN 016]: 100%|██████████| 1759/1759 [00:07<00:00, 231.94it/s, batch_loss=0.7873649] 


[VALID 016]: loss=0.786565363407135, metrics={'mse': 0.786565363407135}


[TRAIN 017]: 100%|██████████| 1759/1759 [00:07<00:00, 228.17it/s, batch_loss=0.4890897] 


[VALID 017]: loss=0.7892701029777527, metrics={'mse': 0.7892701029777527}
🏃 View run blushing-skink-181 at: http://localhost:8080/#/experiments/970024866457651319/runs/7a3fc6d319b34e47a8646513effe3cbb
🧪 View experiment at: http://localhost:8080/#/experiments/970024866457651319
